In [1]:
import os
import random
import matplotlib.pyplot as plt
import cv2
import numpy as np
from typing import List, Tuple

### Задания

1. Для датасета Nails segmentation создать генератор, который на каждой итерации возвращает пару списков из заданного количества (аргумент функции) изображений и масок к ним (итератор должен перемешивать примеры).
2. Добавить в генератор случайную аугментацию (каждая применяется случайно). После преобразований все изображения должны иметь одинаковый размер. Обратите внимание, что большинство преобразований должны применяться одинаково к изображению и маске:
    - A. Поворот на случайный угол
    - B. Отражение по вертикали, горизонтали
    - C. Вырезание части изображения
    - D. Размытие

In [2]:
images_dir = r'C:\Users\Ilya\Tasks PAK\data\nails_segmentation\images'
labels_dir = r'C:\Users\Ilya\Tasks PAK\data\nails_segmentation\labels'

images_files = [file for file in os.listdir(images_dir)]

images_path = [os.path.join(images_dir, file) for file in images_files]
labels_path = [os.path.join(labels_dir, file) for file in images_files]

#### Task1

In [3]:
def nails_segmentation_generator (quantity : int, images_path: List[str], 
                    labels_path: List[str], augmentation_flag : bool ) -> Tuple [List[np.ndarray],List[np.ndarray]] :
    """
    Генератор для создания пар списков изображений и масок к ним.

    :param quantity:  длина возвращаемых списков
    :param images_path: список путей к файлам изображений
    :param labels_path: список путей к файлам масок
    :return: кортеж из двух списков: список изображений и список масок
    """
    pairs_path = list(zip(images_path, labels_path))

    while True:
        random.shuffle(pairs_path)
        
        part_pairs_path = pairs_path [:quantity]
        part_images_path, part_labels_path = zip(*part_pairs_path)

        images = []
        labels = []

        for i in range (quantity):
            image = cv2.imread (part_images_path[i])
            label = cv2.imread (part_labels_path[i])

            if (augmentation_flag) :
                # 1 - Поворот на случайный угол
                # 2 - Отражение по вертикали, горизонтали
                # 3 - Вырезание части изображения
                # 4 - Размытие
                type_of_change = random.randint(1,4)
                if (type_of_change == 1):
                    image, label = rotate(image, label)
                if (type_of_change == 2):
                    image, label = reflection(image, label)
                if (type_of_change == 3):
                    image, label = cutting(image, label)
                if (type_of_change == 3):
                    image = blur(image)
            images.append (image)
            labels.append (label)
        yield images, labels
        

In [4]:
augmentation_flag = True
quantity = 10

generator = nails_segmentation_generator(quantity, images_path, labels_path, augmentation_flag)

images, labels = next(generator)

NameError: name 'rotate' is not defined

In [5]:
for img, label in zip(images, labels):
    cv2.imshow('Image', img)
    cv2.imshow('Label', label)

    key = cv2.waitKey(0) & 0xff
    
    if key == 27:
        break
cv2.destroyAllWindows()

NameError: name 'images' is not defined

#### Task2

In [6]:
np.random.seed(43)

In [11]:
def rotate (img : np.ndarray, label : np.ndarray) -> Tuple [img : np.ndarray , label : np.ndarray] :
     """
    функция для поворота на случайный угол изображения и его маски.

    :param img:  исходное изображение (np.ndarray)
    :param label :  исходная маска (np.ndarray)
    :return: кортеж из двух элементов: повернутое изображение (np.ndarray), повернутая маска (np.ndarray)
    """
     alpha = random.randint(-180, 180)
     if alpha == 0:
          alpha = random.randint(-180, 180)

     
     height, width = img.shape[:2]
     center_h, center_w = height//2, width//2


     rot_mat = cv2.getRotationMatrix2D((center_h, center_w), alpha, 1.0)
     
     img =  cv2.warpAffine(img, rot_mat, (height, width), flags=cv2.INTER_LINEAR)
     label = cv2.warpAffine(label, rot_mat, (height, width), flags=cv2.INTER_LINEAR)

     return img, label


NameError: name 'img' is not defined

In [12]:
def reflection (img : np.ndarray, label : np.ndarray) -> Tuple [img : np.ndarray , label : np.ndarray] :
     """
     функция для отражения изображения вокруг случайно выбранной оси.

     :param img:  исходное изображение (np.ndarray)
     :param label :  исходная маска (np.ndarray)
     :return: кортеж из двух элементов: отраженное изображение (np.ndarray), отраженная маска (np.ndarray)
     """
     # выбираем сценарий отражения
     # Вертикальное отражение изображения (flipCode == 0)
     # Горизонтальное отражение изображения (flipCode == 1)
     # Одновременное горизонтальное и вертикальное отражение изображения (flipCode == -1)
     flipCode = random.randint(-1, 1)

     img = cv2.flip(img,flipCode)
     label = cv2.flip(label, flipCode)
     return img,label 
    

NameError: name 'img' is not defined

In [13]:
def cutting(img : np.ndarray, label : np.ndarray) -> Tuple [img : np.ndarray , label : np.ndarray] :
    """
     функция для вырезания части изображения.

     :param img:  исходное изображение (np.ndarray)
     :param label :  исходная маска (np.ndarray)
     :return: кортеж из двух элементов: часть изображения (np.ndarray), часть маски (np.ndarray)
     """
    bbox = np.random.randint(0, 128, size=4)  # x, y, w, h
    
    crop_height = 128
    crop_width = 128

    height, width = img.shape[:2]

    x = random.randint(0, width - crop_width)
    y = random.randint(0, height - crop_height)

    img = img[y:y+crop_height, x: x+crop_width]
    label = label[y:y+crop_height, x: x+crop_width]

    return img, label

NameError: name 'img' is not defined

In [14]:
def blur(img : np.ndarray) -> np.ndarray :
    """
    функция для размытия изображения.
    :param img:  исходное изображение (np.ndarray)
    :return: размытое изображение
    """
    img = cv2.medianBlur(img, 3)
    return img
    

### Лабороторная 

Ловлю призраков необходимо реализовать через поиск и сопоставление ключевых точек на изображениях. Алгоритм должен состоять из следующих шагов:

- Загрузка изображения, на котором необходимо осуществлять поиск;
- Загрузка изображения(ий) призраков;
- Обнаружить на них ключевые точки и вычислить для них любые понравившиеся вам дескрипторы SIFT, SURF, ORB;
- Сопоставить точки шаблона (призрака) с точками изображения через Brute-Force Matching или FLANN Matching и найти какой области соответстветствует призрак;
- Найти гомографию используя алгоритм RANSAC. Выделить призрака на изображение рамкой.


Ключевые слова для поиска в Google и документации OpenCV: findHomography, RANSAC, SIFT_Create, FlannBasedMatcher.

ЛР 6.1: нужно поймать одного призрака ЛР 6.2: нужно поймать всех призраков

##### 6.1

In [49]:
all_ghost = ['pampkin_ghost.png', 'scary_ghost.png', 'candy_ghost.png']

i = 1 

ghosts_augmentation = []
for _ in range(1):
    temp = cv2.imread(all_ghost[i])
    temp = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
    ghosts_augmentation.append(temp)
    ghosts_augmentation.append(cv2.flip(temp,0))

In [50]:
img = cv2.imread('lab7.png')
total_img = img.copy()
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()

flag = True

In [ ]:
while flag:
    flag = False
    for ghost in ghost_augmentation :
        kp1, dp1 = sift.detectAndCompute(ghost, None)
        kp_img, dp_img = sift.detectAndCompute(gray_img, None)

        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)

        flann = cv2.FlannBasedMatcher(index_params, search_params)
    
        matches = flann.knnMatch(dp1, dp_img, k=2)
        good = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good.append(m)

        if len(good) < 10:
            continue
            
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_img[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        if M is not None:
            h, w = ghost.shape
            pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
            dst = cv2.perspectiveTransform(pts, M)

            gray_img = cv2.fillPoly(gray_img, [np.int32(dst)], 0)

            total_img = cv2.polylines(total_img, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)

            flag = True
        
        cv2.imshow('1', gray_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# Отображение окончательного результата
cv2.imshow('Detected Ghost', total_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


#### 6.2

In [37]:
all_ghost = ['pampkin_ghost.png', 'scary_ghost.png', 'candy_ghost.png']

all_ghosts_augmentation = []
for ghost in all_ghost:
    temp = cv2.imread(ghost)
    temp = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
    all_ghosts_augmentation.append(temp)
    all_ghosts_augmentation.append(cv2.flip(temp,0))

In [ ]:
img = cv2.imread('lab7.png')
total_img = img.copy()
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()

flag = True

In [52]:
while flag:
    flag = False  
    for ghost in all_ghosts_augmentation:
        
        kp1, dp1 = sift.detectAndCompute(ghost, None)
        kp_img, dp_img = sift.detectAndCompute(gray_img, None)

        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)

        flann = cv2.FlannBasedMatcher(index_params, search_params)

       
        matches = flann.knnMatch(dp1, dp_img, k=2)
        good = []
        for m, n in matches:
            if m.distance < 0.65 * n.distance:
                good.append(m)

        if len(good) < 10:
            continue


        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_img[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        if M is not None:
            h, w = ghost.shape
            pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
            dst = cv2.perspectiveTransform(pts, M)

            gray_img = cv2.fillPoly(gray_img, [np.int32(dst)], 0)

            total_img = cv2.polylines(total_img, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)

            flag = True

        cv2.imshow('1', gray_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
cv2.imshow('Detected Ghost', total_img)
cv2.waitKey(0)
cv2.destroyAllWindows()